# Create KAKEN Awards from JSPS Grants-in-Aid Data

Creates KAKEN awards from the JSPS Grants-in-Aid for Scientific Research database. ~1M+ projects.

**Prerequisites:**
- Run `scripts/local/kaken_to_s3.py` to download and upload the data first.
- Note: Full download takes ~6 days due to rate limiting.

**Data source:** https://kaken.nii.ac.jp/  
**S3 location:** `s3a://openalex-ingest/awards/kaken/kaken_projects.parquet`

**JSPS funder details:**
- funder_id: 4320334764
- display_name: Japan Society for the Promotion of Science
- ROR: https://ror.org/00hhkn466
- DOI: 10.13039/501100001691

**Common research categories:**
- Grant-in-Aid for Scientific Research (A/B/C/S)
- Grant-in-Aid for Young Scientists
- Grant-in-Aid for Challenging Research
- Grant-in-Aid for JSPS Fellows
- Grant-in-Aid for Transformative Research Areas

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.kaken_projects_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/kaken/kaken_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~1M+)
SELECT COUNT(*) as total_projects FROM openalex.awards.kaken_projects_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.kaken_projects_raw LIMIT 5;

## Step 2: Create KAKEN Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.kaken_awards
USING delta
AS
WITH
-- Get JSPS funder from OpenAlex
jsps_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320334764  -- Japan Society for the Promotion of Science
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000 as id,

        -- Display name = project title
        g.title as display_name,

        -- Description = abstract
        g.abstract as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Amount in JPY
        g.amount as amount,
        'JPY' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map KAKEN categories to types
        CASE
            WHEN LOWER(g.category) LIKE '%fellow%' THEN 'fellowship'
            WHEN LOWER(g.category) LIKE '%young%' THEN 'fellowship'
            WHEN LOWER(g.category) LIKE '%early-career%' THEN 'fellowship'
            WHEN LOWER(g.category) LIKE '%encouragement%' THEN 'fellowship'
            WHEN LOWER(g.category) LIKE '%start%' THEN 'fellowship'
            WHEN LOWER(g.category) LIKE '%specially promoted%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%scientific research%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%challenging%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%transformative%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%pioneering%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%international%' THEN 'research'
            WHEN LOWER(g.category) LIKE '%publication%' THEN 'other'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = research category
        g.category as funder_scheme,

        -- Provenance
        'kaken' as provenance,

        -- Dates
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    -- NRID is not an ORCID, so we leave orcid NULL
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        COALESCE(g.institution, g.pi_affiliation) as name,
                        'Japan' as country,
                        -- Include NRID as an identifier if available
                        CASE 
                            WHEN g.pi_nrid IS NOT NULL THEN 
                                array(struct(
                                    CONCAT('https://nrid.nii.ac.jp/nrid/', g.pi_nrid) as id,
                                    'nrid' as type,
                                    'kaken' as asserted_by
                                ))
                            ELSE CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>)
                        END as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in scraped data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL
        g.landing_page_url,

        -- No DOI for KAKEN grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.kaken_projects_raw g
    CROSS JOIN jsps_funder f
    WHERE g.project_id IS NOT NULL
      AND TRIM(g.project_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Insert into openalex_awards
INSERT INTO openalex.awards.openalex_awards
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    ARRAY() as funded_outputs,
    0 as funded_outputs_count
FROM openalex.awards.kaken_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count
SELECT COUNT(*) as total_kaken_awards FROM openalex.awards.kaken_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator
FROM openalex.awards.kaken_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be JSPS)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.kaken_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.kaken_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme (research category) distribution - top 20
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.kaken_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_with_amount,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_abstract
FROM openalex.awards.kaken_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.kaken_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 30;

In [ ]:
%sql
-- Check funding amount distribution (in millions JPY)
SELECT 
    CASE 
        WHEN amount < 1000000 THEN '<1M JPY'
        WHEN amount < 5000000 THEN '1-5M JPY'
        WHEN amount < 10000000 THEN '5-10M JPY'
        WHEN amount < 50000000 THEN '10-50M JPY'
        WHEN amount < 100000000 THEN '50-100M JPY'
        ELSE '>100M JPY'
    END as amount_range,
    COUNT(*) as cnt
FROM openalex.awards.kaken_awards
WHERE amount IS NOT NULL
GROUP BY 1
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check lead institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.kaken_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;